In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from pydub import AudioSegment
# from pydub.playback import play

In [ ]:
class AudioFP():
    ## Initializing AudioFP object properties
    def __init__(self, filename):
        self.songname = filename
        self.channels = []
        self.songdata = []
        self.framerate = []
        
    ## Reading audio file using pydub
    def read_audiofile(self):
        audiofile = AudioSegment.from_file(self.songname)
        self.songdata = np.frombuffer(audiofile._data, np.int16)
        for chn in range(audiofile.channels):
            self.channels.append(self.songdata[chn::audiofile.channels])
        self.framerate = audiofile.frame_rate

In [ ]:
song1 = AudioFP("Choc--Eigenvalue-Subspace-Decomposition.mp3")
song1.read_audiofile()

In [ ]:
print(np.shape(song1.songdata))
print(np.shape(song1.channels))

In [ ]:
plt.plot(song1.channels[0])
plt.plot(song1.channels[1])